# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse        Shipments  \
0  Apr 19 2023 12:01PM  261415        19      ELI04202023   
1  Apr 19 2023 11:58AM  261414        19  TR-RIH-04202023   
2  Apr 19 2023 11:48AM  261412        10        MSP220919   
3  Apr 19 2023 11:48AM  261412        10        MSP220921   
4  Apr 19 2023 11:48AM  261412        10        MSP220922   
5  Apr 19 2023 11:48AM  261412        10        MSP220923   
6  Apr 19 2023 11:48AM  261412        10        MSP220924   
7  Apr 19 2023 11:48AM  261412        10        MSP220925   
8  Apr 19 2023 11:48AM  261412        10        MSP220926   
9  Apr 19 2023 11:48AM  261412        10        MSP220927   

                 Customer ShipmentStatus  
0   Eli Lilly and Company       Released  
1  GCH Granules USA, Inc.       Released  
2        Methapharm, Inc.       Released  
3        Methapharm, Inc.       Released  
4        Methapharm, Inc.       Released  
5        Methapharm, Inc.       Released  
6        Methapharm, Inc.       Released  
7        Methapharm, Inc.       Released  
8        Methapharm, Inc.       Released  
9        Methapharm, Inc.       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
24  261406      Executing          1
25  261410       Released          9
26  261412       Released         11
27  261414       Released          1
28  261415       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
261406                NaN        1.0       NaN
261410                NaN        NaN       9.0
261412                NaN        NaN      11.0
261414                NaN        NaN       1.0
261415                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
260869                0.0        7.0      26.0
260910                0.0        9.0       4.0
261117                0.0        9.0       1.0
261321                0.0       13.0      12.0
261343                0.0        0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
260869                  0          7        26
260910                  0          9         4
261117                  0          9         1
261321                  0         13        12
261343                  0          0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               260869          0          7        26
1               260910          0          9         4
2               261117          0          9         1
3               261321          0         13        12
4               261343          0          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               260869                   7       26
1               260910                   9        4
2               261117                   9        1
3               261321                  13       12
4               261343                            1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                         Customer
0   Apr 19 2023 12:01PM  261415        19            Eli Lilly and Company
1   Apr 19 2023 11:58AM  261414        19           GCH Granules USA, Inc.
2   Apr 19 2023 11:48AM  261412        10                 Methapharm, Inc.
13  Apr 19 2023 11:08AM  261406        20         Granules Consumer Health
14  Apr 19 2023 11:07AM  261404        15  Brookfield Pharmaceuticals, LLC
19  Apr 19 2023 11:07AM  261402        15      Virtus Pharmaceuticals, LLC
24  Apr 19 2023 11:06AM  261401        15  Brookfield Pharmaceuticals, LLC
26  Apr 19 2023 11:05AM  261410        20      Reliable 1 Laboratories LLC
35  Apr 19 2023 10:39AM  261405        19           NAPP Technologies, LLC
37  Apr 19 2023 10:26AM  261403        10                      Emerginnova

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                         Customer  \
0  Apr 19 2023 12:01PM  261415        19            Eli Lilly and Company   
1  Apr 19 2023 11:58AM  261414        19           GCH Granules USA, Inc.   
2  Apr 19 2023 11:48AM  261412        10                 Methapharm, Inc.   
3  Apr 19 2023 11:08AM  261406        20         Granules Consumer Health   
4  Apr 19 2023 11:07AM  261404        15  Brookfield Pharmaceuticals, LLC   
5  Apr 19 2023 11:07AM  261402        15      Virtus Pharmaceuticals, LLC   
6  Apr 19 2023 11:06AM  261401        15  Brookfield Pharmaceuticals, LLC   
7  Apr 19 2023 11:05AM  261410        20      Reliable 1 Laboratories LLC   
8  Apr 19 2023 10:39AM  261405        19           NAPP Technologies, LLC   
9  Apr 19 2023 10:26AM  261403        10                      Emerginnova   

  Completed Executing Released  
0                            1  
1                            1  
2                           11  
3                   1           
4                   3        2  
5                            5  
6                            2  
7                            9  
8                            2  
9                   1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                         Customer  \
0  Apr 19 2023 12:01PM  261415        19            Eli Lilly and Company   
1  Apr 19 2023 11:58AM  261414        19           GCH Granules USA, Inc.   
2  Apr 19 2023 11:48AM  261412        10                 Methapharm, Inc.   
3  Apr 19 2023 11:08AM  261406        20         Granules Consumer Health   
4  Apr 19 2023 11:07AM  261404        15  Brookfield Pharmaceuticals, LLC   
5  Apr 19 2023 11:07AM  261402        15      Virtus Pharmaceuticals, LLC   
6  Apr 19 2023 11:06AM  261401        15  Brookfield Pharmaceuticals, LLC   
7  Apr 19 2023 11:05AM  261410        20      Reliable 1 Laboratories LLC   
8  Apr 19 2023 10:39AM  261405        19           NAPP Technologies, LLC   
9  Apr 19 2023 10:26AM  261403        10                      Emerginnova   

  Released Executing Completed  
0        1                      
1        1                      
2       11                      
3                  1            
4        2         3            
5        5                      
6        2                      
7        9                      
8        2                      
9                  1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                         Customer  \
0  Apr 19 2023 12:01PM  261415        19            Eli Lilly and Company   
1  Apr 19 2023 11:58AM  261414        19           GCH Granules USA, Inc.   
2  Apr 19 2023 11:48AM  261412        10                 Methapharm, Inc.   
3  Apr 19 2023 11:08AM  261406        20         Granules Consumer Health   
4  Apr 19 2023 11:07AM  261404        15  Brookfield Pharmaceuticals, LLC   

  Released Executing Completed  
0        1                      
1        1                      
2       11                      
3                  1            
4        2         3

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                         Customer  \
0  Apr 19 2023 12:01PM  261415        19            Eli Lilly and Company   
1  Apr 19 2023 11:58AM  261414        19           GCH Granules USA, Inc.   
2  Apr 19 2023 11:48AM  261412        10                 Methapharm, Inc.   
3  Apr 19 2023 11:08AM  261406        20         Granules Consumer Health   
4  Apr 19 2023 11:07AM  261404        15  Brookfield Pharmaceuticals, LLC   

   Released  Executing  Completed  
0       1.0        NaN        NaN  
1       1.0        NaN        NaN  
2      11.0        NaN        NaN  
3       NaN        1.0        NaN  
4       2.0        3.0        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                         Customer  \
0  Apr 19 2023 12:01PM  261415        19            Eli Lilly and Company   
1  Apr 19 2023 11:58AM  261414        19           GCH Granules USA, Inc.   
2  Apr 19 2023 11:48AM  261412        10                 Methapharm, Inc.   
3  Apr 19 2023 11:08AM  261406        20         Granules Consumer Health   
4  Apr 19 2023 11:07AM  261404        15  Brookfield Pharmaceuticals, LLC   

   Released  Executing  Completed  
0       1.0        0.0        0.0  
1       1.0        0.0        0.0  
2      11.0        0.0        0.0  
3       0.0        1.0        0.0  
4       2.0        3.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         1045553      12.0        9.0        0.0
15         1306894      35.0       23.0        7.0
19         1829752      34.0       15.0        0.0
20         1305712      40.0       26.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  1045553      12.0        9.0        0.0
1        15  1306894      35.0       23.0        7.0
2        19  1829752      34.0       15.0        0.0
3        20  1305712      40.0       26.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      12.0        9.0        0.0
1        15      35.0       23.0        7.0
2        19      34.0       15.0        0.0
3        20      40.0       26.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse     Status  Value
0        10   Released   12.0
1        15   Released   35.0
2        19   Released   34.0
3        20   Released   40.0
4        10  Executing    9.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    15    19    20
Status                           
Completed   0.0   7.0   0.0   0.0
Executing   9.0  23.0  15.0  26.0
Released   12.0  35.0  34.0  40.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    15    19    20
0          Completed   0.0   7.0   0.0   0.0
1          Executing   9.0  23.0  15.0  26.0
2           Released  12.0  35.0  34.0  40.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    15    19    20
0  Completed   0.0   7.0   0.0   0.0
1  Executing   9.0  23.0  15.0  26.0
2   Released  12.0  35.0  34.0  40.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()